---
format:
    html:
        embed-resources: true
---

# Cleaning: Part-1: 

The goal now is to post process the job descriptions you collected in the Crawl

In this section you MUST do it manually using traditional data-science cleaning and parsing skills, i.e. without use of ANY LLM tools or wrappers.

The unit of analysis in this case is "job" 

Your job is to write a cleaning script to Parse the Crawl results and create a data-frame which will be saved to `data/processed-jobs-1.csv`, this data-frame will be used for later EDA. 


You should try to extract as many features as possible, here a possible suggestions. 

These features can provide a comprehensive overview when analyzing job postings for trends, patterns, and insights.

You will almost certainly not be able to extract all of these features, but do your best to make the most detailed data set that you can. You can also ADD more features to this list if they come to mind.

- **Job Title**
- **Company Name**
- **Sector/Industry**
- **Location** (City, State, Country)
- **Job Type** (Full-time, Part-time, Contract, Internship)
- **Salary** (Range or Exact)
- **Experience Level** (Entry-level, Mid-level, Senior-level)
- **Education Requirements** (Degree, Major, Certifications)
- **Skills/Technologies Required** (e.g., Python, SQL, Machine Learning, Cloud)
- **Job Responsibilities/Duties**
- **Required Years of Experience**
- **Benefits** (Health insurance, Retirement plans, Paid time off)
- **Remote Work Options** (Remote, Hybrid, On-site)
- **Application Deadline**
- **Job Posting Date**
- **Job Description Length** (Number of words or characters)
- **Keywords/Frequency of Terms**
- **Certifications Required or Preferred** (e.g., AWS Certified, PMP)
- **Team Size** (If mentioned)
- **Company Size** (Small, Medium, Large)
- **Company Reputation/Ranking** (If available)
- **Job Posting Platform** (Where the job was posted, e.g., LinkedIn, Indeed)
- **Company Values or Culture** (Diversity, Innovation, Sustainability)
- **Visa Sponsorship Availability**
- **Interview Process Information** (If mentioned)
- **Expected Start Date**
- **Job Posting Expiry Date**
- **Gender Diversity Language** (If any)
- **Working Hours/Shift Type** (Day shift, Night shift, Flexible hours)
- **Required Language Skills**
- **Job Location Proximity to Major Cities** (If provided)
- **Travel Requirements** (Percentage or Frequency)
- **Team Collaboration Tools Mentioned** (Slack, Zoom, etc.)
- **Reporting Line** (e.g., Reports to Senior Manager)
- **Job Benefits Related to Learning & Development** (e.g., Courses, Training)
- **Company Stock Options** (If offered)
- **Required Soft Skills** (e.g., Communication, Leadership)
- **Company Perks** (Gym membership, Free meals, Company car)
- **Job Posting ID or Reference Number**


While cleaning try to address the following as best you can

- **Missing Data**: Identify missing or null values in features like salary, company name, or location.
- **Duplicates**: Check for duplicate job listings or job descriptions.
- **Inconsistent Formatting**: Ensure consistency in categorical variables (e.g., job titles, location formats).
- **Data Type Validation**: Ensure each feature has the appropriate data type (e.g., salary as numeric, dates as datetime).
- **Outliers**: Detect outliers in salary, years of experience, or job description length.
